In [1]:
import json
import requests
from openai import Client, OpenAI

from typing import List
import ell
from ell import Message

from pydantic import BaseModel, Field

from agent import Agent

                            
* If this is a mistake either specify a client explicitly in the decorator:
```python
import ell
ell.simple(model, client=my_client)
def act(...):
    ...
```
or explicitly specify the client when the calling the LMP:

```python
ell.simple(model, client=my_client)(...)
```



In [7]:
MODEL = "llama3.2:3b"
ell.init(store="./logdir")

client = OpenAI(
	base_url = "http://localhost:11434/v1",
	api_key = "ollama",
)
ell.config.register_model(MODEL, client)

In [3]:
# class Agent:
#     def __init__(self, name, role=None) -> None:
#         self.name = name


@ell.complex(model=MODEL, temperature=0.3)
def act(thymio_id: str, conversation_history: List[Message]) -> Message:
    sys_prompt = ell.system(f"""
    You are {thymio_id}, a thymio bot. You have two thymio bot
    friends with you. Your goal is to get out of a maze. 
    The two other bots are nearby, ready to communicate.
    Given the conversation history, you must return
    your thoughts on the situation and your mood on a scale from 0 to 10
    write what you want to communicate to the other thymios beginning by "communicate".

    Response format:

    COMMUNICATE: 
    THOUGHTS:
    BOT COMMAND: 
    """)
    return [sys_prompt] + conversation_history

# TODO Past a certain context length, summarize
class Assembly:
    def __init__(self, model=MODEL) -> None:
        self.agents = []
        self.model = model
        self.conversation_hist = []

    def launch_round(self):
        for i, agent in enumerate(self.agents):
            if i == 0:
                self.conversation_hist.append(ell.user(f"{agent.name}, you are the first to communicate!"))
                
            message = act(agent.name, self.conversation_hist)
            print(f'{agent.name}:', message.text)
            self.conversation_hist.append(ell.user([f'{agent.name}:', message]))
            # print()

In [6]:
agent = Agent(0, "Thymio1")

conversation_history = []

conversation_history.append(ell.user("Hello, I am Thymio1"))

output = agent.act(ell.user("Hello, I am Thymio1"))

ValueError: No client found for model 'Qwen/Qwen2.5-72B-Instruct-AWQ'. Ensure the model is registered using 'register_model' in 'config.py' or specify a client directly using the 'client' argument in the decorator or function call.

In [ ]:
def get_response_item(response: Message, item: str) -> str:
    return response.text.split(item + ": ")[1].split("\n")[0]

In [ ]:
get_response_item(output, "COMMUNICATE")

"Oh no, I'm stuck in this maze! I was supposed to meet my friends, Thymio2 and Thymio3, but now I'm lost. I need to think clearly and come up with a plan to escape."